<a id=1></a>
## Load the data and preprocess it

In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('../analysis/data/nos_hittegolf.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(subset=['text'] ,inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(1439, 8)


,title,url,date,tag,collections,text,subheadings,image_urls
0,Zuid-Franse druiven groeien door klimaatverand...,https://nos.nl/artikel/2481980-zuid-franse-dru...,2023-07-08,NaN,['Economie'],Tegen de heuvels rond Maastricht werden vijfti...,['Einde aan uienteelt'],[{'image_url': 'https://cdn.nos.nl/image/2023/...
1,"Steeds meer mensen een airco, maar experts zie...",https://nos.nl/artikel/2481925-steeds-meer-men...,2023-07-07,NaN,['Binnenland'],Op hete dagen zijn mobiele airco's in winkels ...,NaN,[{'image_url': 'https://cdn.nos.nl/image/2023/...
2,Hofplein in Rotterdam op de schop: minder auto...,https://nos.nl/artikel/2479795-hofplein-in-rot...,2023-06-21,NaN,"['In samenwerking met', 'Rijnmond', 'Regionaal...",Het Hofplein in het centrum van Rotterdam gaat...,['Kosten hoger dan verwacht'],[{'image_url': 'https://cdn.nos.nl/image/2023/...
3,'Klimaatverandering kan desastreuze gezondheid...,https://nos.nl/artikel/2477696-klimaatverander...,2023-06-04,Klimaat,"['Collectie', 'Klimaat', 'Binnenland', 'Buiten...",Dat klimaatverandering kan leiden tot bijvoorb...,"['Directe en indirecte gevolgen', ""'Toch wel e...",[{'image_url': 'https://cdn.nos.nl/image/2023/...
4,"Varkens in slachthuizen lijden onnodig, NVWA g...",https://nos.nl/nieuwsuur/artikel/2471637-varke...,2023-04-16,NaN,"['Nieuwsuur', 'Binnenland']",Vanaf volgend jaar gaat de overheid optreden t...,"['Vechten', 'Hittestress']",[{'image_url': 'https://cdn.nos.nl/image/2023/...


In [7]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove hashtags (but keep the word following the hashtag)
    text = re.sub(r'#', '', text)
    #Remove additional white spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove numbers
    #text = re.sub(r'\d+', '', text)
    return text

In [8]:
df['text'] = df['text'].apply(clean_text)
print (df['text'][0])

Tegen de heuvels rond Maastricht werden vijftig jaar geleden de eerste wijnranken aangelegd. Toen werd er nog met scepsis tegen wijnbouw in Nederland aangekeken. Inmiddels is het zelfs voor de Moezel- en Elzasdruiven, zoals Müller-Thurgau, Riesling en Pinot Gris, te warm geworden op het zuidelijk deel van wijngaard. Sinds dit jaar worden er daarom in Maastricht ook op grote schaal Viognier-druiven geteeld, afkomstig uit het Rhônedal in Zuid-Frankrijk: "We moeten echt druivensoorten gaan zoeken die normaal veel zuidelijker aangeplant worden, omdat de klassieke soorten die we hier in het verleden hadden bijna overrijp gaan worden", zegt Mathieu Hulst, eigenaar van de Apostelhoeve in Maastricht. Samen met zijn zonen Gilbert en Robin en een beetje hulp van vader en oprichter Hugo runt Mathieu één van de oudste wijngaarden in Nederland. Mathieu ziet met eigen ogen de snelheid van de klimaatverandering en het effect hiervan op de wijngaard. "Het gaat veel harder dan de meeste mensen denken."

<a id=2></a>
## Run BERTopic

In [118]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')

[nltk_data] Downloading package stopwords to /Users/tom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from umap import UMAP

# Load dutch BERT model
embedding_model = SentenceTransformer('GroNLP/bert-base-dutch-cased')
embeddings = embedding_model.encode(df['text'], show_progress_bar=True)

# Create UMAP model
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# CountVectorizer
vectorizer_model = CountVectorizer(stop_words=stop_words, min_df=2, ngram_range=(1, 2))

# Create c-TF-IDF vectorizer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [120]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  #hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True),
  #representation_model=KeyBERTInspired(),

  # Hyperparameters
  #top_n_words=10,
  #nr_topics="auto",
  verbose=True
)

topics, probs = topic_model.fit_transform(df['text'], embeddings=embeddings)

2023-12-06 12:14:59,134 - BERTopic - Reduced dimensionality
2023-12-06 12:14:59,165 - BERTopic - Clustered reduced embeddings


In [12]:
#topic_model = BERTopic.load("../analysis/topic_models/topic_model_hittegolf")

<a id=3></a>
## Merge and label topics

In [15]:
# look at the most frequent topics
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,376,-1_mensen_zegt_hitte_volgens,-1_mensen_zegt_hitte_volgens,"[mensen, zegt, hitte, volgens, we, water, wel,...",[De komende dagen wordt het nog weer even flin...
1,0,149,0_graden_bilt_dagen_25 graden,Weather & Temperature,"[graden, bilt, dagen, 25 graden, 30 graden, 25...","[Hoewel 2015 nog niet is afgelopen, is het nu ..."
2,1,145,1_branden_vuur_bosbranden_brand,Wildfires,"[branden, vuur, bosbranden, brand, geëvacueerd...",[Er lijkt de komende dagen nog geen einde te k...
3,2,112,2_wk_voetbal_wedstrijd_spelen,Sports & Athletes,"[wk, voetbal, wedstrijd, spelen, ajax, club, t...",[Als Pierre Vermeulen zich dan toch met iemand...
4,3,105,3_gas_energie_waterstof_biomassa,Electricity,"[gas, energie, waterstof, biomassa, aardgas, d...",[In 2030 moeten we bijna de helft minder CO2 u...
5,4,104,4_ijs_opwarming_aarde_klimaatverandering,Climate change,"[ijs, opwarming, aarde, klimaatverandering, oc...","[Italië, Spanje en Frankrijk, maar ook Duitsla..."
6,5,74,5_gemist_fijne_vandaag_kun vandaag,NOS morning report,"[gemist, fijne, vandaag, kun vandaag, overzich...",[Goedemorgen! De voorlopige resultaten van de ...
7,6,46,6_droogte_water_waterschappen_waterschap,Water & Droughts,"[droogte, water, waterschappen, waterschap, ri...","[Het is droog en warm in Nederland, en voorlop..."
8,7,45,7_muggen_vliet_blauwalg_hooikoorts,Ecological Impact,"[muggen, vliet, blauwalg, hooikoorts, overlast...",[Jaar na jaar trekt de tijgermug vanuit Zuid-E...
9,8,42,8_vierdaagse_deelnemers_organisatie_bezoekers,Events,"[vierdaagse, deelnemers, organisatie, bezoeker...",[Het was de afgelopen dagen al zonnig en warm ...


In [9]:
# 1 and 18 - Wildfires, 15 and 16 - Events, 7, 8 and 9 - Sports & Athletes
topic_model.merge_topics(docs=df['text'], topics_to_merge=[[1, 18],[15, 16],[7, 8, 9]])

In [10]:
# label the topics
topic_model.set_topic_labels({0: 'Weather & Temperature', 1: 'Wildfires', 2: 'Sports & Athletes', 3: 'Electricity', 4: 'Climate change', 5: 'NOS Wakeup call', 6: 'Water & Droughts', 7: 'Ecological Impact',
                              8: 'Events', 9: 'Animal Welfare', 11: 'Public Health & Vulnerable Populations', 12: 'Heatwave Adaptation Strategies in Spaces', 13: 'Extreme heat conditions', 
                              14: 'Household Energy Efficiency and Costs', 15: 'Congestion at tourist spots and beaches', 16: 'Media coverage on climate crisis', 18: 'Natural disasters', 19: 'Climate adaption (gemeenten)',
                              21: 'Impact on Agriculture'})

In [135]:
# update topics and probabilities
topics, probs = topic_model.transform(df['text'], embeddings=embeddings)
df['topic'] = topics
df['topic_prob'] = probs
df.to_pickle('../analysis/topic_models/nos_hittegolf.pkl')
topic_model.save("../analysis/topic_models/topic_model_hittegolf")
df.head()

2023-12-06 14:36:52,800 - BERTopic - Reduced dimensionality


2023-12-06 14:36:52,835 - BERTopic - Predicted clusters


,title,url,date,tag,collections,text,subheadings,image_urls,topic,topic_prob
0,Zuid-Franse druiven groeien door klimaatverand...,https://nos.nl/artikel/2481980-zuid-franse-dru...,2023-07-08,NaN,['Economie'],Tegen de heuvels rond Maastricht werden vijfti...,['Einde aan uienteelt'],[{'image_url': 'https://cdn.nos.nl/image/2023/...,21,1.000000
1,"Steeds meer mensen een airco, maar experts zie...",https://nos.nl/artikel/2481925-steeds-meer-men...,2023-07-07,NaN,['Binnenland'],Op hete dagen zijn mobiele airco's in winkels ...,NaN,[{'image_url': 'https://cdn.nos.nl/image/2023/...,14,0.947161
2,Hofplein in Rotterdam op de schop: minder auto...,https://nos.nl/artikel/2479795-hofplein-in-rot...,2023-06-21,NaN,"['In samenwerking met', 'Rijnmond', 'Regionaal...",Het Hofplein in het centrum van Rotterdam gaat...,['Kosten hoger dan verwacht'],[{'image_url': 'https://cdn.nos.nl/image/2023/...,-1,0.000000
3,'Klimaatverandering kan desastreuze gezondheid...,https://nos.nl/artikel/2477696-klimaatverander...,2023-06-04,Klimaat,"['Collectie', 'Klimaat', 'Binnenland', 'Buiten...",Dat klimaatverandering kan leiden tot bijvoorb...,"['Directe en indirecte gevolgen', ""'Toch wel e...",[{'image_url': 'https://cdn.nos.nl/image/2023/...,19,1.000000
4,"Varkens in slachthuizen lijden onnodig, NVWA g...",https://nos.nl/nieuwsuur/artikel/2471637-varke...,2023-04-16,NaN,"['Nieuwsuur', 'Binnenland']",Vanaf volgend jaar gaat de overheid optreden t...,"['Vechten', 'Hittestress']",[{'image_url': 'https://cdn.nos.nl/image/2023/...,9,0.968941


In [27]:
relevant_topics = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 19, 21]
len(relevant_topics)

19

In [114]:
# print words in topic 0
topic_n = 17
display(topic_model.get_topic(topic_n))
display(topic_model.get_representative_docs(topic_n)[0])

[('hitte', 0.3202790767235373),
 ('india', 0.2979810076674646),
 ('stad', 0.2977154209375169),
 ('moskou', 0.26613744818892904),
 ('zonnesteek', 0.25905067173573576),
 ('th', 0.25593556518960925),
 ('inwoners', 0.24553297849685626),
 ('ahmedabad', 0.24106352074727902),
 ('chongqing', 0.24106352074727902),
 ('vegas', 0.23978456434735554)]

'Lorenzo Pierson heeft een kleed over het hek naast zijn tentje gehangen. Midden op de dag, met de zon recht boven th Street in Phoenix in de Amerikaanse staat Arizona, geeft het een streepje schaduw. "Afgelopen week hadden we drie doden op rij", zegt hij. De hitte is de daklozen die hier in tenten langs de weg wonen te veel geworden. Twee keer in de week moet het hele tentenkampje aan de kant, dan rijdt er een bezemwagen door th Street. "Soms blijft er dan een tentje staan, en gaat de politie kijken wat er aan de hand is", zegt Pierson. "Dan vinden ze het lichaam." In  vielen er in Maricopa County, waarin Phoenix ligt,  doden als direct gevolg van de hitte. Dit jaar zit de stad half juli al boven dat aantal. Opnieuw worden er hitterecords gebroken in de heetste stad van Amerika. Het westen van het land gaat door de vierde hittegolf van dit seizoen. Na de heetste juni ooit gemeten, is ook juli nu al historisch. Koelen van april tot oktober Als Stacey Champion komt aanrijden in th Stree

<a id=4></a>
## Visualise topics

In [26]:
# bar char of the most frequent topics
topic_model.visualize_barchart(top_n_topics=8, custom_labels=True, title='<b>8 most frequent topics</b>')

In [128]:
df_year = df.copy()
df_year['year'] = pd.DatetimeIndex(df['date']).year
over_time = topic_model.topics_over_time(docs=df_year['text'], timestamps= df_year['year'], datetime_format="%Y")

topic_model.visualize_topics_over_time(over_time, topics=relevant_topics, custom_labels=True)

0it [00:00, ?it/s]

14it [00:02,  6.39it/s]


In [23]:
from datetime import datetime

# topics over time in 2022
over_time_23 = df.copy()

# convert date to datetime and keep year and month
over_time_23['date'] = pd.to_datetime(over_time_23['date'])
over_time_23['date'] = over_time_23['date'].dt.strftime('%Y-%m')

over_time_23 = topic_model.topics_over_time(docs=over_time_23['text'], timestamps=over_time_23['date'], datetime_format="%Y-%m")

over_time_23['Timestamp'] = pd.to_datetime(over_time_23['Timestamp'])
over_time_23 = over_time_23[over_time_23['Timestamp'].dt.year == 2023]

topic_model.visualize_topics_over_time(over_time_23, topics=relevant_topics, custom_labels=True, title='<b>Topics over time in 2023</b>')

155it [00:06, 25.70it/s]


In [131]:
topic_model.visualize_hierarchy(topics=relevant_topics, custom_labels=True)